# HU Extension              Assignment 10      E63 Big Data Analytics 		                  	
## Handed out: 04/08/2023                         Due by 12:00 noon EST on Saturday, 04/15/2023
### Emmanuel Aboah


__Note For TF__: In problem 3 we are asked to "Treat horsepower as a feature and mpg as the target variable (label)". I instead constructed my vector with all fields. But i have accommodated for it in problem 4 since I essentially solved problem 4 in problem 3. This notebook has many cells so i did not order it in the interest of time.

# Problem 1. 

On slides 70 to 71 of lecture notes on Spark ML you will find results of the predictions of Linear Regression model for the train set and the test set generated from the simple-ml dataset.  Capture outputs of those two prediction exercises into two DataFrames and use them to assess the accuracy of prediction in those two cases. Express errors as percent of correct answers when compared to the total numbers of samples. Those numbers are not identical in two cases. Attached ZIP file simple-ml.zip contains the artificial data set used in the lecture notes.

Set up Linear regression models

In [7]:
# Imports
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

findspark.init()

# Setup spark configs and context
spark = SparkSession.builder.appName("ml_app").getOrCreate()
sc = SparkContext.getOrCreate()
sc.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble


23/04/15 13:19:28 WARN Utils: Your hostname, LAPTOP-85L1BUVJ resolves to a loopback address: 127.0.1.1; using 172.19.201.226 instead (on interface eth0)
23/04/15 13:19:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 13:19:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Read data
simple_ml_df = spark.read.json("./data/simple-ml.json")

simple_ml_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- lab: string (nullable = true)
 |-- value1: long (nullable = true)
 |-- value2: double (nullable = true)



Exploratory Data Analysis

In [15]:
simple_ml_df.select(simple_ml_df["color"]).distinct().show()

+-----+
|color|
+-----+
|green|
|  red|
| blue|
+-----+



In [18]:
simple_ml_df.select(simple_ml_df["value1"]).distinct().show()

+------+
|value1|
+------+
|     1|
|    12|
|     8|
|    35|
|     2|
|    15|
|    16|
|    45|
+------+



In [19]:
simple_ml_df.select(simple_ml_df["value2"]).distinct().show()

+------------------+
|            value2|
+------------------+
|14.386294994851129|
| 38.97187133755819|
+------------------+



In [20]:
# Select feature from the DF using RFormula

from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ .")

In [21]:
# Fit the RFormula transformer

fitted_rformula = supervised.fit(simple_ml_df)

In [22]:
# Apply the transformations to the DF

prepared_df = fitted_rformula.transform(simple_ml_df)
prepared_df.show(10, False)

+-----+----+------+------------------+---------------------------------+-----+
|color|lab |value1|value2            |features                         |label|
+-----+----+------+------------------+---------------------------------+-----+
|green|good|1     |14.386294994851129|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|blue |bad |8     |14.386294994851129|[0.0,0.0,8.0,14.386294994851129] |0.0  |
|blue |bad |12    |14.386294994851129|[0.0,0.0,12.0,14.386294994851129]|0.0  |
|green|good|15    |38.97187133755819 |[0.0,1.0,15.0,38.97187133755819] |1.0  |
|green|good|12    |14.386294994851129|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|green|bad |16    |14.386294994851129|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|red  |good|35    |14.386294994851129|[1.0,0.0,35.0,14.386294994851129]|1.0  |
|red  |bad |1     |38.97187133755819 |[1.0,0.0,1.0,38.97187133755819]  |0.0  |
|red  |bad |2     |14.386294994851129|[1.0,0.0,2.0,14.386294994851129] |0.0  |
|red  |bad |16    |14.386294994851129|[1.0,0.0,16.0,

Classification with logistic linear regression

In [23]:
# Random split to train and test

train, test = prepared_df.randomSplit([0.7, 0.3])

In [24]:
# Instantiate Classifier

from pyspark.ml.classification import LogisticRegression

logistic_regr = LogisticRegression(labelCol="label", featuresCol="features")

In [27]:
# Explain parameter

logistic_regr.explainParams()

"aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)\nelasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)\nfamily: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)\nfeaturesCol: features column name. (default: features, current: features)\nfitIntercept: whether to fit an intercept term. (default: True)\nlabelCol: label column name. (default: label, current: label)\nlowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)\nlowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimizat

In [28]:
# Explain individual parameter
logistic_regr.explainParam("predictionCol")

'predictionCol: prediction column name. (default: prediction)'

In [33]:
# Train the model by fitting the logistic_regression train dataset

fitted_logistic_regr_model = logistic_regr.fit(train)

In [48]:
# Prediction with train set

train_prediction_df = (
    fitted_logistic_regr_model
    .transform(train)
    .select("label", "prediction")
)

train_prediction_df.show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
+-----+----------+
only showing top 20 rows



In [49]:
# Prediction with test set

test_prediction_df = (
    fitted_logistic_regr_model
    .transform(test)
    .select("label", "prediction")
)

test_prediction_df.show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+



Express errors as percent of correct answers when compared to the total numbers of samples.

---> Training set

In [58]:
# Incorrect predictions
wrong_cnt = (
    train_prediction_df
    .selectExpr("(label == prediction) as wrong")
    .filter("wrong == false")
    .count()
)

print(f"Incorrect count: {wrong_cnt}")

Incorrect count: 15


In [60]:
# Correct predications
corr_cnt = (
    train_prediction_df
    .selectExpr("(label == prediction) as correct")
    .filter("correct == true")
    .count()
)

print(f"Correct count: {corr_cnt}")

Correct count: 76


In [61]:
# total number dataset

total_sample = prepared_df.count()

print(f"Sample size: {total_sample}")

Sample size: 110


In [78]:
# percentage of incorrect predictions compared to sample size

print(
    f"percentage: %.2f error rate in {total_sample} sample size" % (wrong_cnt / corr_cnt)
)

percentage: 0.20 error rate in 110 sample size


---> Testing set

In [82]:
# Incorrect predictions
test_wrong_cnt = (
    test_prediction_df
    .selectExpr("(label == prediction) as wrong")
    .filter("wrong == false")
    .count()
)

print(f"Incorrect count: {test_wrong_cnt}")

Incorrect count: 5


In [81]:
# Correct predications
test_corr_cnt = (
    test_prediction_df
    .selectExpr("(label == prediction) as correct")
    .filter("correct == true")
    .count()
)

print(f"Correct count: {test_corr_cnt}")

Correct count: 14


In [83]:
# percentage of incorrect predictions compared to sample size

print(
    f"percentage: %.2f error rate in {total_sample} sample size" % (test_wrong_cnt / test_corr_cnt)
)

percentage: 0.36 error rate in 110 sample size


In [98]:
# Area under ROC

print(fitted_logistic_regr_model.summary.areaUnderROC)

0.9023668639053255


In [99]:
# ROC

fitted_logistic_regr_model.summary.roc.show()

+-------------------+-------------------+
|                FPR|                TPR|
+-------------------+-------------------+
|                0.0|                0.0|
|                0.0|0.20512820512820512|
|                0.0| 0.4358974358974359|
|0.17307692307692307| 0.4358974358974359|
|0.17307692307692307| 0.6666666666666666|
|0.17307692307692307| 0.8461538461538461|
|0.17307692307692307|                1.0|
| 0.3269230769230769|                1.0|
| 0.4807692307692308|                1.0|
| 0.6730769230769231|                1.0|
| 0.8653846153846154|                1.0|
|                1.0|                1.0|
|                1.0|                1.0|
+-------------------+-------------------+



In [100]:
# Precicion Recal curve

fitted_logistic_regr_model.summary.pr.show()

+-------------------+-------------------+
|             recall|          precision|
+-------------------+-------------------+
|                0.0|                1.0|
|0.20512820512820512|                1.0|
| 0.4358974358974359|                1.0|
| 0.4358974358974359| 0.6538461538461539|
| 0.6666666666666666| 0.7428571428571429|
| 0.8461538461538461| 0.7857142857142857|
|                1.0|             0.8125|
|                1.0| 0.6964285714285714|
|                1.0|           0.609375|
|                1.0|  0.527027027027027|
|                1.0| 0.4642857142857143|
|                1.0|0.42857142857142855|
+-------------------+-------------------+



# Problem 2. 

Attached file auto_mpg_original.csv contains a set of data on automobile characteristics and fuel consumption. File auto_mpg_description.csv contains the description of the data.  Import data into Spark. Randomly select 20% of your data for testing and use the remaining data for training. Find all null values in all numerical columns. Write an Estimator that will replace nulls, if any, with average values for respective column.

In [101]:
# Create schema for DF
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

schema = StructType ([
    StructField("mpg", IntegerType(), True),
    StructField("cylinders", IntegerType(), True),
    StructField("displacement", IntegerType(), True),
    StructField("horsepower", IntegerType(), True),
    StructField("weight", IntegerType(), True),
    StructField("acceleration", IntegerType(), True),
    StructField("model year", IntegerType(), True),
    StructField("origin", IntegerType(), True),
    StructField("car name", StringType(), True),
])

auto_df = (
    spark.read
    .option("inferSchema", False)
    .option("header", False)
    .schema(schema=schema)
    .csv("./data/auto_mpg_original.csv")
)

Exploratory Data Analysis

In [102]:
# Show data

auto_df.show()

+---+---------+------------+----------+------+------------+----------+------+---------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin| car name|
+---+---------+------------+----------+------+------------+----------+------+---------+
| 18|        8|         307|       130|  3504|          12|        70|     1|chevrolet|
| 15|        8|         350|       165|  3693|        null|        70|     1|    buick|
| 18|        8|         318|       150|  3436|          11|        70|     1| plymouth|
| 16|        8|         304|       150|  3433|          12|        70|     1|      amc|
| 17|        8|         302|       140|  3449|        null|        70|     1|     ford|
| 15|        8|         429|       198|  4341|          10|        70|     1|     ford|
| 14|        8|         454|       220|  4354|           9|        70|     1|chevrolet|
| 14|        8|         440|       215|  4312|        null|        70|     1| plymouth|
| 14|        8|         455|    

In [104]:
# count of all records

auto_df.count()

406

```bash 
# Confirm number of rows in csv
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw10_sparl_ML$ cat ./data/auto_mpg_original.csv | wc -l
406
```

In [103]:
# distinct car by make

auto_df.select("car name").distinct().show()

+---------+
| car name|
+---------+
|    buick|
|  pontiac|
| mercedes|
|   toyota|
|     saab|
|      amc|
|       vw|
|  peugeot|
| chrysler|
| plymouth|
|vokswagen|
|  citroen|
|    chevy|
|     audi|
|   datsun|
|      bmw|
|    dodge|
|     ford|
|  toyouta|
|    capri|
+---------+
only showing top 20 rows



In [110]:
# Count null, none, NaN in all columns of dataframe
from pyspark.sql.functions import col, isnan, when, count

# Null count
auto_df.select(
    [count(when(col(c).isNull(), c))
     .alias(c) for c in auto_df.columns]
    ).show()

+---+---------+------------+----------+------+------------+----------+------+--------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|car name|
+---+---------+------------+----------+------+------------+----------+------+--------+
|140|        0|           1|         1|     0|         282|         0|     0|       0|
+---+---------+------------+----------+------+------------+----------+------+--------+



In [111]:
# NaN check
auto_df.select(
    [count(when(isnan(c), c))
     .alias(c) for c in auto_df.columns]
    ).show()

+---+---------+------------+----------+------+------------+----------+------+--------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|car name|
+---+---------+------------+----------+------+------------+----------+------+--------+
|  0|        0|           0|         0|     0|           0|         0|     0|       0|
+---+---------+------------+----------+------+------------+----------+------+--------+



In [133]:
# get columns
num_cols = auto_df.columns[:8]

print(num_cols)

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']


In [134]:
# Use Imputer as transformer and fill null values with mean

from pyspark.ml.feature import Imputer

output_cols = ["{}_imputed".format(c) for c in num_cols]

imputer = Imputer(
    inputCols=num_cols,
    outputCols=output_cols,
    strategy="mean"
)

i_auto_df = imputer.fit(auto_df).transform(auto_df)

i_auto_df.show(5)

+---+---------+------------+----------+------+------------+----------+------+---------+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin| car name|mpg_imputed|cylinders_imputed|displacement_imputed|horsepower_imputed|weight_imputed|acceleration_imputed|model year_imputed|origin_imputed|
+---+---------+------------+----------+------+------------+----------+------+---------+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+
| 18|        8|         307|       130|  3504|          12|        70|     1|chevrolet|         18|                8|                 307|               130|          3504|                  12|                70|             1|
| 15|        8|         350|       165|  3693|        null|        70|     1|    buick| 

In [142]:
# Show only imputed columns
imputed_cols = [c for c in i_auto_df.columns if c.endswith("imputed")]

i_auto_df.select(imputed_cols).show(5)

+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+
|mpg_imputed|cylinders_imputed|displacement_imputed|horsepower_imputed|weight_imputed|acceleration_imputed|model year_imputed|origin_imputed|
+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+
|         18|                8|                 307|               130|          3504|                  12|                70|             1|
|         15|                8|                 350|               165|          3693|                  15|                70|             1|
|         18|                8|                 318|               150|          3436|                  11|                70|             1|
|         16|                8|                 304|               150|          3433|                  12|                70|             1|
|     

# Problem 3. 

Look initially at two variables in the data set from the previous problem: mpg  (miles per gallon) and the horsepower. Treat horsepower as a feature and mpg as the target variable (label). Use Spark ML linear regression to identify the model for the relationship. Use the test data to illustrate accuracy of the linear regression model and its ability to predict the relationship. Calculate two standard measures of model accuracy. Create a diagram using any technique of convenience to presents the model (straight line), and the original test data. Please label your axes and use different colors for original data and the predicted data.

In [201]:
# VectorAssembler to transform model for training
from pyspark.ml.feature import VectorAssembler

vector_asmblr = VectorAssembler(inputCols=imputed_cols, outputCol="features")

v_auto_df = vector_asmblr.transform(i_auto_df)

v_auto_df.show(5)

+---+---------+------------+----------+------+------------+----------+------+---------+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+--------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin| car name|mpg_imputed|cylinders_imputed|displacement_imputed|horsepower_imputed|weight_imputed|acceleration_imputed|model year_imputed|origin_imputed|            features|
+---+---------+------------+----------+------+------------+----------+------+---------+-----------+-----------------+--------------------+------------------+--------------+--------------------+------------------+--------------+--------------------+
| 18|        8|         307|       130|  3504|          12|        70|     1|chevrolet|         18|                8|                 307|               130|          3504|                  12|                70|             1|[18.0,8.0,307.0,1...|
| 15

In [203]:
# select columns for training

v_mpg_hp_df = v_auto_df.select(["mpg_imputed", "horsepower_imputed", "features"])

v_mpg_hp_df.show()

+-----------+------------------+--------------------+
|mpg_imputed|horsepower_imputed|            features|
+-----------+------------------+--------------------+
|         18|               130|[18.0,8.0,307.0,1...|
|         15|               165|[15.0,8.0,350.0,1...|
|         18|               150|[18.0,8.0,318.0,1...|
|         16|               150|[16.0,8.0,304.0,1...|
|         17|               140|[17.0,8.0,302.0,1...|
|         15|               198|[15.0,8.0,429.0,1...|
|         14|               220|[14.0,8.0,454.0,2...|
|         14|               215|[14.0,8.0,440.0,2...|
|         14|               225|[14.0,8.0,455.0,2...|
|         15|               190|[15.0,8.0,390.0,1...|
|         26|               115|[26.0,4.0,133.0,1...|
|         19|               165|[19.0,8.0,350.0,1...|
|         27|               153|[27.0,8.0,351.0,1...|
|         28|               175|[28.0,8.0,383.0,1...|
|         25|               175|[25.0,8.0,360.0,1...|
|         15|               

In [204]:
# Split the dataset 20% for testing and 80% for training

splits = v_mpg_hp_df.randomSplit([0.8, 0.2])

train_df = splits[0]
test_df = splits[1]

In [207]:
# Training data

train_df.show(5, False)

+-----------+------------------+-------------------------------------------+
|mpg_imputed|horsepower_imputed|features                                   |
+-----------+------------------+-------------------------------------------+
|10         |200               |[10.0,8.0,307.0,200.0,4376.0,15.0,70.0,1.0]|
|10         |215               |[10.0,8.0,360.0,215.0,4615.0,14.0,70.0,1.0]|
|11         |210               |[11.0,8.0,318.0,210.0,4382.0,15.0,70.0,1.0]|
|12         |160               |[12.0,8.0,350.0,160.0,4456.0,15.0,72.0,1.0]|
|12         |167               |[12.0,8.0,400.0,167.0,4906.0,15.0,73.0,1.0]|
+-----------+------------------+-------------------------------------------+
only showing top 5 rows



In [208]:
# Testing data
test_df.show(5, False)

+-----------+------------------+-------------------------------------------+
|mpg_imputed|horsepower_imputed|features                                   |
+-----------+------------------+-------------------------------------------+
|9          |193               |[9.0,8.0,304.0,193.0,4732.0,15.0,70.0,1.0] |
|11         |150               |[11.0,8.0,400.0,150.0,4997.0,14.0,73.0,1.0]|
|11         |180               |[11.0,8.0,350.0,180.0,3664.0,11.0,73.0,1.0]|
|11         |208               |[11.0,8.0,429.0,208.0,4633.0,11.0,72.0,1.0]|
|12         |225               |[12.0,8.0,455.0,225.0,4951.0,11.0,73.0,1.0]|
+-----------+------------------+-------------------------------------------+
only showing top 5 rows



In [210]:
# Setup linear regression for relationship between mpg and horsepower
from pyspark.ml.regression import LinearRegression

linear_regr = LinearRegression(featuresCol="features", labelCol="mpg_imputed")

linear_regr_model = linear_regr.fit(train_df)

print("Coefficients: " + str(linear_regr_model.coefficients))
print("Intercept: " + str(linear_regr_model.intercept))

Coefficients: [1.0000000000000016,4.3323909353699874e-14,-1.6569210397435787e-15,7.73463128862864e-16,5.849309233204158e-17,1.1072331678973232e-14,-1.8355335626351814e-14,-1.972575627495115e-14]
Intercept: 1.0542032102911864e-12


Use the test data to illustrate accuracy of the linear regression model and its ability to predict the relationship.

In [211]:
# Predictions with test data

from pyspark.sql.functions import round

lr_predictions = linear_regr_model.transform(test_df)
prediction_res = lr_predictions.select(round("prediction").alias("predictions"), "mpg_imputed", "features")

prediction_res.show(10, False)

+-----------+-----------+-------------------------------------------+
|predictions|mpg_imputed|features                                   |
+-----------+-----------+-------------------------------------------+
|9.0        |9          |[9.0,8.0,304.0,193.0,4732.0,15.0,70.0,1.0] |
|11.0       |11         |[11.0,8.0,400.0,150.0,4997.0,14.0,73.0,1.0]|
|11.0       |11         |[11.0,8.0,350.0,180.0,3664.0,11.0,73.0,1.0]|
|11.0       |11         |[11.0,8.0,429.0,208.0,4633.0,11.0,72.0,1.0]|
|12.0       |12         |[12.0,8.0,455.0,225.0,4951.0,11.0,73.0,1.0]|
|13.0       |13         |[13.0,8.0,302.0,129.0,3169.0,12.0,75.0,1.0]|
|13.0       |13         |[13.0,8.0,302.0,140.0,4294.0,16.0,72.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,145.0,4055.0,12.0,76.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,150.0,4699.0,15.0,74.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,165.0,4274.0,12.0,72.0,1.0]|
+-----------+-----------+-------------------------------------------+
only showing top 10 

Calculate two standard measures of model accuracy.

In [212]:
# Linear regregression evaluator r2

from pyspark.ml.evaluation import RegressionEvaluator

lr_eval_r2 = RegressionEvaluator(predictionCol="predictions", 
                              labelCol="mpg_imputed",
                              metricName="r2")

print("R Squared (R2) on test data = %g" % lr_eval_r2.evaluate(prediction_res))

R Squared (R2) on test data = 1


In [213]:
# Linear regregression evaluator Root Mean Squared Erro (RMSE)

from pyspark.ml.evaluation import RegressionEvaluator

lr_eval_rmse = RegressionEvaluator(predictionCol="predictions", 
                              labelCol="mpg_imputed",
                              metricName="rmse")

print("Root Mean Squared Error (RMSE) on test data = %g" % lr_eval_rmse.evaluate(prediction_res))

Root Mean Squared Error (RMSE) on test data = 0


In [214]:
train_df.describe().show()

+-------+-----------------+------------------+
|summary|      mpg_imputed|horsepower_imputed|
+-------+-----------------+------------------+
|  count|              318|               318|
|   mean|21.70754716981132|104.35220125786164|
| stddev|5.817357332206395| 37.77580190611737|
|    min|               10|                46|
|    max|               44|               230|
+-------+-----------------+------------------+



Create a diagram using any technique of convenience to presents the model (straight line), and the original test data. Please label your axes and use different colors for original data and the predicted data.

In [ ]:
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

min_max_scaler = preprocessing.MinMaxScaler()
_columns = ["mpg_imputed", "predictions"]
x = prediction_res.columns[1]
y = prediction_res.columns[0]

x = pd.DataFrame(data=min_max_scaler.fit_transform(x), columns=_columns)

fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))

index = 0
axs = axs.flatten()

for i, k in enumerate(_columns):
    sns.regplot(y=y, x=x[k], ax=axs[i],scatter_kws={"s": 60} )
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

# Problem 4. 

Consider the data in auto_mpg_original.csv. Usually, we are interested in miles per gallon (mpg) and would like to be able to predict that quantity based on other features.  Use Linear Regression to make the predictions. Assess accuracy of your model. Identify the most important feature. 

Already covered this in problem 3 so i will make another transformation here for horsepower as feature.

In [231]:
# new df with only mpg and horsepower

mpg_hp_df = i_auto_df.selectExpr(
    "mpg_imputed as mpg",
    "horsepower_imputed as horsepower"
    )

In [234]:
# Use RFormula to transform

from pyspark.ml.feature import RFormula
supervised = RFormula(formula="mpg ~ .")

In [235]:
# fit

fitted_mpg_hp_df = supervised.fit(mpg_hp_df)

In [236]:
prep_mpg_hp_df = fitted_mpg_hp_df.transform(mpg_hp_df)

prep_mpg_hp_df.show(10, False)

+---+----------+--------+-----+
|mpg|horsepower|features|label|
+---+----------+--------+-----+
|18 |130       |[130.0] |18.0 |
|15 |165       |[165.0] |15.0 |
|18 |150       |[150.0] |18.0 |
|16 |150       |[150.0] |16.0 |
|17 |140       |[140.0] |17.0 |
|15 |198       |[198.0] |15.0 |
|14 |220       |[220.0] |14.0 |
|14 |215       |[215.0] |14.0 |
|14 |225       |[225.0] |14.0 |
|15 |190       |[190.0] |15.0 |
+---+----------+--------+-----+
only showing top 10 rows



In [240]:
# test and train data 

test_II, train_II = prep_mpg_hp_df.randomSplit([0.2, 0.8])

In [241]:
linear_regr_II = LinearRegression(featuresCol="features", labelCol="mpg")

linear_regr_model_II = linear_regr_II.fit(train_II)

print("Coefficients: " + str(linear_regr_model_II.coefficients))
print("Intercept: " + str(linear_regr_model_II.intercept))

Coefficients: [-0.1056907566444211]
Intercept: 32.63070856012571


In [242]:
# Test predictions

lr_predictions_II = linear_regr_model_II.transform(test_II)
prediction_res_II = lr_predictions_II.select(round("prediction").alias("predictions"), "mpg", "features")

prediction_res.show(10, False)

+-----------+-----------+-------------------------------------------+
|predictions|mpg_imputed|features                                   |
+-----------+-----------+-------------------------------------------+
|9.0        |9          |[9.0,8.0,304.0,193.0,4732.0,15.0,70.0,1.0] |
|11.0       |11         |[11.0,8.0,400.0,150.0,4997.0,14.0,73.0,1.0]|
|11.0       |11         |[11.0,8.0,350.0,180.0,3664.0,11.0,73.0,1.0]|
|11.0       |11         |[11.0,8.0,429.0,208.0,4633.0,11.0,72.0,1.0]|
|12.0       |12         |[12.0,8.0,455.0,225.0,4951.0,11.0,73.0,1.0]|
|13.0       |13         |[13.0,8.0,302.0,129.0,3169.0,12.0,75.0,1.0]|
|13.0       |13         |[13.0,8.0,302.0,140.0,4294.0,16.0,72.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,145.0,4055.0,12.0,76.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,150.0,4699.0,15.0,74.0,1.0]|
|13.0       |13         |[13.0,8.0,350.0,165.0,4274.0,12.0,72.0,1.0]|
+-----------+-----------+-------------------------------------------+
only showing top 10 

In [243]:
train_II.describe().show()

+-------+------------------+------------------+------------------+
|summary|               mpg|        horsepower|             label|
+-------+------------------+------------------+------------------+
|  count|               327|               327|               327|
|   mean|21.507645259938837|105.24159021406727|21.507645259938837|
| stddev|  5.88221059580541| 37.68394209662769|  5.88221059580541|
|    min|                 9|                48|               9.0|
|    max|                44|               225|              44.0|
+-------+------------------+------------------+------------------+



In [244]:
trainingSummary_II = linear_regr_model_II.summary
print("RMSE: %f" % trainingSummary_II.rootMeanSquaredError)
print("r2: %f" % trainingSummary_II.r2)

RMSE: 4.322043
r2: 0.458464


# Problem 5. 

Repeat the previous analysis using Decision Tree Regression. Compare the precision of the Decision Tree Regression and Linear Regression. It is alright if DTR turns out worse than the Linear regression. Usually it is better. Decision Tree Regression model usually has featureImportance attribute. Please read the documentation and find out whether such attribute exists. If it does, tell us which are the most important features.

In [245]:
# Decision Tree

from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol="features", labelCol="mpg_imputed")

dt_model = dt.fit(train_df)
dt_predicts = dt_model.transform(test_df)

dt_evals = RegressionEvaluator(
     labelCol="mpg_imputed", predictionCol="prediction", metricName="rmse"
)

# RMSE
rmse = dt_evals.evaluate(dt_predicts)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.205069


In [247]:
# r2

dt_evals_II = RegressionEvaluator(
     labelCol="mpg_imputed", predictionCol="prediction", metricName="r2"
)

r2 = dt_evals.evaluate(dt_predicts)

print("R Squared (r^2) on test data = %g" % r2)

R Squared (r^2) on test data = 0.205069


In [252]:
# Show Predictions

dt_predicts.select("horsepower_imputed", "mpg_imputed", round("prediction").alias("predictions")).show(10)

+------------------+-----------+-----------+
|horsepower_imputed|mpg_imputed|predictions|
+------------------+-----------+-----------+
|               193|          9|       10.0|
|               150|         11|       10.0|
|               180|         11|       10.0|
|               208|         11|       10.0|
|               225|         12|       12.0|
|               129|         13|       13.0|
|               140|         13|       13.0|
|               145|         13|       13.0|
|               150|         13|       13.0|
|               165|         13|       13.0|
+------------------+-----------+-----------+
only showing top 10 rows



In [253]:
# feature importance

dt_model.featureImportances

SparseVector(8, {0: 1.0})